#  Clean and Split Dataset into train/val/test from the Clevr folder

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
%cd /content/gdrive/MyDrive/clevr-dataset-gen/image_generation

/content/gdrive/MyDrive/clevr-dataset-gen/image_generation


In [3]:
import glob,os,shutil,tqdm,joblib,json

In [4]:
import torch,json
def extract_infos(path):
  dictionary={
    "cube": 0,
    "sphere": 1,
    "triangular pyramid": 2,
    "cylinder": 3,
    "triangular prism": 4,
    "cone": 5,
    "toroid": 6,
    "square pyramid": 7,
    "star": 8,
    "diamond": 9,
    "gray": 10,
    "red": 11,
    "blue":12,
    "green": 13,
    "brown": 14,
    "purple": 15,
    "cyan": 16,
    "yellow": 17,
    "orange":18,
    "pink": 19
  }
  with open(path) as fd:
            objects = json.load(fd)["objects"]

  targets=[]
  boxes=torch.FloatTensor([])
  labels=torch.LongTensor([])
  lista_concepts=torch.LongTensor([])
  ys=[]
  for obj in objects:
    objbox=torch.FloatTensor([obj["x"],obj["y"],obj["x"]+obj["width"],obj["y"]+obj["height"]]).view(1,4)
    ys.append(obj["y"])
    boxes=torch.concatenate([boxes,objbox])
    concepts=torch.zeros((1,20)).long()
    concepts[0,dictionary[obj["shape"]]]=1
    concepts[0,dictionary[obj["color"]]]=1
    lista_concepts=torch.concatenate([lista_concepts,concepts])

  labels=torch.ones(len(objects)).long()
  targets={"boxes":boxes,"labels":labels,"concepts":lista_concepts}
  return targets

In [5]:
def bb_intersection_over_union(boxA, boxB):
	# determine the (x, y)-coordinates of the intersection rectangle
	xA = max(boxA[0], boxB[0])
	yA = max(boxA[1], boxB[1])
	xB = min(boxA[2]+boxA[0], boxB[2]+boxB[0])
	yB = min(boxA[3]+boxA[1], boxB[3]+ boxB[1])
	# compute the area of intersection rectangle
	interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
	# compute the area of both the prediction and ground-truth
	# rectangles
	boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
	boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
	# compute the intersection over union by taking the intersection
	# area and dividing it by the sum of prediction + ground-truth
	# areas - the interesection area
	iou = interArea / float(boxAArea + boxBArea - interArea)
	# return the intersection over union value
	return iou

In [ ]:
base_path="/content/gdrive/MyDrive/clevr-dataset-gen/image_generation/dueoggetti"

for task in range(1,6):
  print(task)
  path_imgs=os.path.join(base_path,"image_task"+str(task))
  path_scenes=os.path.join(base_path,"scene_task"+str(task))
  lista_immagini= glob.glob(os.path.join(path_imgs,"*"))
  lista_scene= [os.path.join(path_scenes,"CLEVR_new_"+img.split(os.sep)[-1].split("_")[-1].split(".")[0]+".json") for img in lista_immagini]

  for img,scene in zip(lista_immagini,lista_scene):
    with open(scene) as fd:
      objs=json.load(fd)["objects"]
    bb1=objs[0]["x"],objs[0]["y"],objs[0]["width"],objs[0]["height"]
    bb2=objs[1]["x"],objs[1]["y"],objs[1]["width"],objs[1]["height"]
    iou=bb_intersection_over_union(bb1,bb2)
    print(iou,img)
    if iou!= 0:
      os.remove(img)
      os.remove(scene)


In [ ]:
for task in range(1,6):
  path_imgs=os.path.join(base_path,"image_task"+str(task))
  path_scenes=os.path.join(base_path,"scene_task"+str(task))
  lista_immagini= glob.glob(os.path.join(path_imgs,"*"))
  print(task,len(lista_immagini))

1 1640
2 1715
3 1720
4 1729
5 1670


In [ ]:
base_path="/content/gdrive/MyDrive/clevr-dataset-gen/image_generation/dueoggetti"

for task in range(1,6):
  print(task)
  path_imgs=os.path.join(base_path,"image_task"+str(task))
  path_scenes=os.path.join(base_path,"scene_task"+str(task))
  lista_immagini= glob.glob(os.path.join(path_imgs,"*"))
  lista_scene= [os.path.join(path_scenes,"CLEVR_new_"+img.split(os.sep)[-1].split("_")[-1].split(".")[0]+".json") for img in lista_immagini]
  test_img=lista_immagini[:500]
  test_scene=lista_scene[:500]
  val_img=lista_immagini[500:600]
  val_scene=lista_scene[500:600]
  train_img= lista_immagini[600:]
  train_scene= lista_scene[600:]
  dest_path="/content/gdrive/MyDrive/clevr-dataset-gen/image_generation/clevr_continual_no-Occl"
  for split,data in zip(["train","test","val"],[(train_img,train_scene),(test_img,test_scene),(val_img,val_scene)]):
    imgs,scenes=data
    path_imgs_dest=os.path.join(dest_path,"image_task"+str(task),split)
    path_scenes_dest=os.path.join(dest_path,"scene_task"+str(task),split)
    os.makedirs(path_imgs_dest,exist_ok=True)
    os.makedirs(path_scenes_dest,exist_ok=True)
    print(path_scenes_dest,path_imgs_dest)
    

    for idx in tqdm.tqdm(range(len(imgs))):
      shutil.copy(imgs[idx],os.path.join(path_imgs_dest,str(idx)+".png"))
      targets=extract_infos(scenes[idx])
      joblib.dump(targets,os.path.join(path_scenes_dest,str(idx)+".joblib"))
      #shutil.copy(scenes[idx],os.path.join(path_scenes_dest,str(idx)+".json"))

# ZIP folder

In [ ]:
!zip -r clevr_continual_no-Occl.zip clevr_continual_no-Occl

# remove imgs with overlapped objects

In [6]:
base_path="/content/gdrive/MyDrive/clevr-dataset-gen/image_generation/dueoggetti"


path_imgs=os.path.join(base_path,"image_TESTv2")
path_scenes=os.path.join(base_path,"scene_TESTv2")
lista_immagini= glob.glob(os.path.join(path_imgs,"*"))
lista_scene= [os.path.join(path_scenes,"CLEVR_new_"+img.split(os.sep)[-1].split("_")[-1].split(".")[0]+".json") for img in lista_immagini]

for img,scene in zip(lista_immagini,lista_scene):
  with open(scene) as fd:
    objs=json.load(fd)["objects"]
  bb1=objs[0]["x"],objs[0]["y"],objs[0]["width"],objs[0]["height"]
  bb2=objs[1]["x"],objs[1]["y"],objs[1]["width"],objs[1]["height"]
  iou=bb_intersection_over_union(bb1,bb2)
  print(iou,img)
  if iou!= 0:
    os.remove(img)
    os.remove(scene)

0.0 /content/gdrive/MyDrive/clevr-dataset-gen/image_generation/dueoggetti/image_TESTv2/CLEVR_new_0.png
0.0 /content/gdrive/MyDrive/clevr-dataset-gen/image_generation/dueoggetti/image_TESTv2/CLEVR_new_1.png
0.0 /content/gdrive/MyDrive/clevr-dataset-gen/image_generation/dueoggetti/image_TESTv2/CLEVR_new_2.png
0.0 /content/gdrive/MyDrive/clevr-dataset-gen/image_generation/dueoggetti/image_TESTv2/CLEVR_new_3.png
0.0 /content/gdrive/MyDrive/clevr-dataset-gen/image_generation/dueoggetti/image_TESTv2/CLEVR_new_4.png
0.0 /content/gdrive/MyDrive/clevr-dataset-gen/image_generation/dueoggetti/image_TESTv2/CLEVR_new_5.png
0.0 /content/gdrive/MyDrive/clevr-dataset-gen/image_generation/dueoggetti/image_TESTv2/CLEVR_new_6.png
0.0 /content/gdrive/MyDrive/clevr-dataset-gen/image_generation/dueoggetti/image_TESTv2/CLEVR_new_7.png
0.0 /content/gdrive/MyDrive/clevr-dataset-gen/image_generation/dueoggetti/image_TESTv2/CLEVR_new_8.png
0.0 /content/gdrive/MyDrive/clevr-dataset-gen/image_generation/dueoggetti

# prepare out of distribution dataset

In [14]:
import glob
print(len(glob.glob(os.path.join(base_path,"image_TESTv2","*"))))
lista_immagini=glob.glob(os.path.join(base_path,"image_TESTv2","*"))
lista_scene=glob.glob(os.path.join(base_path,"scene_TESTv2","*"))
dest_path="/content/gdrive/MyDrive/clevr-dataset-gen/image_generation/clevr_TEST"

for split,data in zip(["test_ood"],[(lista_immagini,lista_scene)]):
    imgs,scenes=data
    path_imgs_dest=os.path.join(dest_path,"image_TEST")
    path_scenes_dest=os.path.join(dest_path,"scene_TEST")
    os.makedirs(path_imgs_dest,exist_ok=True)
    os.makedirs(path_scenes_dest,exist_ok=True)
    print(path_scenes_dest,path_imgs_dest)
    

    for idx in tqdm.tqdm(range(len(imgs))):
      shutil.copy(imgs[idx],os.path.join(path_imgs_dest,str(idx)+".png"))
      targets=extract_infos(scenes[idx])
      joblib.dump(targets,os.path.join(path_scenes_dest,str(idx)+".joblib"))
      #shutil.copy(scenes[idx],os.path.join(path_scenes_dest,str(idx)+".json"))

336
/content/gdrive/MyDrive/clevr-dataset-gen/image_generation/clevr_TEST/scene_TEST /content/gdrive/MyDrive/clevr-dataset-gen/image_generation/clevr_TEST/image_TEST


100%|██████████| 336/336 [00:08<00:00, 41.25it/s]


In [15]:
!zip -r clevr_TEST.zip clevr_TEST

updating: clevr_TEST/ (stored 0%)
updating: clevr_TEST/image_TEST/ (stored 0%)
updating: clevr_TEST/image_TEST/0.png (deflated 0%)
updating: clevr_TEST/image_TEST/1.png (deflated 0%)
updating: clevr_TEST/image_TEST/2.png (deflated 0%)
updating: clevr_TEST/image_TEST/3.png (deflated 0%)
updating: clevr_TEST/image_TEST/4.png (deflated 0%)
updating: clevr_TEST/image_TEST/5.png (deflated 0%)
updating: clevr_TEST/image_TEST/6.png (deflated 0%)
updating: clevr_TEST/image_TEST/7.png (deflated 0%)
updating: clevr_TEST/image_TEST/8.png (deflated 0%)
updating: clevr_TEST/image_TEST/9.png (deflated 0%)
updating: clevr_TEST/image_TEST/10.png (deflated 0%)
updating: clevr_TEST/image_TEST/11.png (deflated 0%)
updating: clevr_TEST/image_TEST/12.png (deflated 0%)
updating: clevr_TEST/image_TEST/13.png (deflated 0%)
updating: clevr_TEST/image_TEST/14.png (deflated 0%)
updating: clevr_TEST/image_TEST/15.png (deflated 0%)
updating: clevr_TEST/image_TEST/16.png (deflated 0%)
updating: clevr_TEST/image_TES

In [ ]:
!zip --help

In [ ]:
import itertools

In [ ]:
setup=[i for i in range(10)]



In [ ]:
comb=[i for i in itertools.product(*[setup,setup,setup,setup]) if ((i[1]>(i[0])and (i[3]>i[2]) and ((i[2]!= i[0]) or (i[3]!= i[0]) or (i[2]!= i[1]) or (i[3]!= i[1]))))]

In [ ]:
def diagonal_area(s1,s2,c1,c2):
  return (s1==s2 or s2==c1 or s1 == c1 or s2==c2) and  (s1==s2+1 or s2==c1+1 or s1 == c1+1 or s2==c2+1) and (s1+1==s2 or s2+1==c1 or s1+1 == c1 or s2+1==c2)


In [ ]:

seen_colors=[i for i in itertools.product(*[setup,setup,setup,setup]) if not (diagonal_area(i[0],i[1],i[2],i[3]))]

len(seen_colors)

9754